# EMSN 2.0 - Distance & Quality CNN Training
## Maximale kwaliteit classifier voor 29.000+ detecties

### Wat dit model doet:
- **Distance classificatie:** very_close (TUIN), close (NABIJ), medium (NABIJ), far (VER), very_far (VER)
- **Quality classificatie:** excellent, good, fair, poor, very_poor
- **Multi-task learning:** Eén model voor beide taken

### Data:
- 29.419 gelabelde detecties uit EMSN PostgreSQL (gebalanceerd)
- Labels gebaseerd op wiskundige analyse (RMS, SNR, spectral metrics)
- Audio: 3-seconde MP3 fragmenten van BirdNET-Pi
- **Data staat al op Google Drive!**

### Colab Pro+ Instellingen:
1. Runtime → Change runtime type
2. Hardware accelerator: **GPU**
3. GPU type: **A100** (40GB)
4. High-RAM: **✓ Aan**

### Verwachte tijd: ~1-2 uur met A100

In [ ]:
# === CELL 1: GPU & Environment Check ===
!nvidia-smi

import torch
import gc
import psutil

torch.cuda.empty_cache()
gc.collect()

print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

ram_gb = psutil.virtual_memory().total / 1e9
print(f"RAM: {ram_gb:.1f} GB")
if ram_gb < 20:
    print("⚠️ Enable High-RAM in Runtime settings!")
else:
    print("✅ High RAM beschikbaar")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"GPU Memory: {gpu_mem:.1f} GB")
    
    # Stability for A100
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True
    
    if 'A100' in gpu_name:
        GPU_TYPE = 'A100'
        BATCH_SIZE = 128  # A100 kan veel aan
        print(f"\n🚀 A100 gedetecteerd - Maximum performance mode")
    elif 'V100' in gpu_name:
        GPU_TYPE = 'V100'
        BATCH_SIZE = 64
    else:
        GPU_TYPE = 'T4'
        BATCH_SIZE = 32
else:
    GPU_TYPE = 'CPU'
    BATCH_SIZE = 16
    print("⚠️ Geen GPU!")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✅ Device: {device}")

In [ ]:
# === CELL 2: Mount Google Drive ===
from google.colab import drive

drive.mount('/content/drive')

# Check of training data aanwezig is
import os
from pathlib import Path

DRIVE_DATA_DIR = Path('/content/drive/MyDrive/EMSN/distance_quality_training')

if DRIVE_DATA_DIR.exists():
    csv_path = DRIVE_DATA_DIR / 'training_data.csv'
    audio_dir = DRIVE_DATA_DIR / 'audio'
    
    if csv_path.exists():
        print(f"✅ training_data.csv gevonden")
    else:
        print(f"❌ training_data.csv NIET gevonden")
    
    if audio_dir.exists():
        audio_count = len(list(audio_dir.glob('*.mp3')))
        print(f"✅ Audio directory gevonden: {audio_count:,} MP3 files")
    else:
        print(f"❌ Audio directory NIET gevonden")
else:
    print(f"❌ Data directory niet gevonden: {DRIVE_DATA_DIR}")
    print("\nZorg ervoor dat je prepare_distance_quality_training.py hebt gedraaid!")

In [ ]:
# === CELL 3: Install Dependencies ===
!pip install librosa scikit-learn scikit-image matplotlib tqdm audiomentations pandas numpy -q
print("✅ Dependencies geïnstalleerd")

In [ ]:
# === CELL 4: Configuration ===
from pathlib import Path

# Storage paths
DRIVE_DATA_DIR = Path('/content/drive/MyDrive/EMSN/distance_quality_training')
AUDIO_DIR = DRIVE_DATA_DIR / 'audio'
CSV_PATH = DRIVE_DATA_DIR / 'training_data.csv'

# Local working directories
WORK_DIR = Path('/content/EMSN-Distance-Quality')
MODELS_DIR = WORK_DIR / 'models'
CACHE_DIR = WORK_DIR / 'cache'

for d in [MODELS_DIR, CACHE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Training Configuration - MAXIMAAL voor Pro+
VERSION = '2026_ultimate'
EPOCHS = 100  # Meer epochs, early stopping bepaalt wanneer te stoppen
LEARNING_RATE = 0.001
MIN_LR = 0.00001
PATIENCE = 15  # Early stopping patience
WEIGHT_DECAY = 0.01

# Data parameters
SAMPLE_RATE = 48000
SEGMENT_DURATION = 3.0
N_MELS = 128
N_FFT = 2048
HOP_LENGTH = 512
FMIN = 500
FMAX = 12000  # Hoger voor vogels

# Augmentation
USE_AUGMENTATION = True
AUGMENTATION_FACTOR = 3  # 3x data door augmentation

# Classes
DISTANCE_CLASSES = ['very_close', 'close', 'medium', 'far', 'very_far']
QUALITY_CLASSES = ['excellent', 'good', 'fair', 'poor', 'very_poor']

# Nederlandse labels voor display (zoals op Ulanzi)
DISTANCE_NL = {
    'very_close': 'TUIN',
    'close': 'NABIJ',
    'medium': 'NABIJ',
    'far': 'VER',
    'very_far': 'VER'
}

print(f"📊 CONFIGURATIE:")
print(f"   GPU: {GPU_TYPE}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Epochs: {EPOCHS} (met early stopping)")
print(f"   Augmentation: {USE_AUGMENTATION} ({AUGMENTATION_FACTOR}x)")
print(f"   Data: {DRIVE_DATA_DIR}")

In [ ]:
# === CELL 5: Load Training Data from Google Drive ===
import pandas as pd

print(f"📥 Loading training data from Google Drive...")
df = pd.read_csv(CSV_PATH)
print(f"✅ Loaded {len(df):,} records")

# Show distribution
print(f"\n📊 Distance verdeling:")
print(df['distance_category'].value_counts())

print(f"\n📊 Quality verdeling:")
print(df['quality_category'].value_counts())

print(f"\n📊 Station verdeling:")
print(df['station'].value_counts())

# Check audio files
print(f"\n🔍 Checking audio files...")
existing_files = 0
missing_files = 0
for audio_file in df['audio_file'].dropna():
    if (AUDIO_DIR / audio_file).exists():
        existing_files += 1
    else:
        missing_files += 1

print(f"   ✅ Audio files aanwezig: {existing_files:,}")
print(f"   ❌ Audio files ontbreken: {missing_files:,}")

In [ ]:
# === CELL 6: Spectrogram Generation Functions ===
import librosa
import numpy as np
from skimage.transform import resize

def load_audio(path, sr=SAMPLE_RATE):
    """Load and preprocess audio."""
    try:
        audio, _ = librosa.load(str(path), sr=sr, mono=True)
        return audio
    except Exception as e:
        return None

def augment_audio(audio, sr):
    """Apply audio augmentations."""
    augmented = [audio]  # Original
    
    if not USE_AUGMENTATION:
        return augmented
    
    # Pitch shift
    try:
        augmented.append(librosa.effects.pitch_shift(audio, sr=sr, n_steps=2))
        augmented.append(librosa.effects.pitch_shift(audio, sr=sr, n_steps=-2))
    except:
        pass
    
    # Time stretch
    try:
        stretched = librosa.effects.time_stretch(audio, rate=0.9)
        if len(stretched) > len(audio):
            stretched = stretched[:len(audio)]
        else:
            stretched = np.pad(stretched, (0, len(audio) - len(stretched)))
        augmented.append(stretched)
    except:
        pass
    
    # Add noise
    noise = np.random.normal(0, 0.005, len(audio))
    augmented.append(audio + noise)
    
    # Volume changes
    augmented.append(audio * 0.8)
    augmented.append(audio * 1.2)
    
    return augmented[:AUGMENTATION_FACTOR + 1]

def audio_to_spectrogram(audio, sr=SAMPLE_RATE):
    """Convert audio to mel spectrogram."""
    mel_spec = librosa.feature.melspectrogram(
        y=audio, sr=sr,
        n_mels=N_MELS, n_fft=N_FFT, hop_length=HOP_LENGTH,
        fmin=FMIN, fmax=FMAX
    )
    mel_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Normalize to 0-1
    mel_norm = (mel_db - mel_db.min()) / (mel_db.max() - mel_db.min() + 1e-8)
    
    # Resize to fixed shape
    target_shape = (128, 128)
    if mel_norm.shape != target_shape:
        mel_norm = resize(mel_norm, target_shape, anti_aliasing=True)
    
    return mel_norm.astype(np.float32)

def process_audio_file(args):
    """Process single audio file to spectrograms."""
    path, distance_label, quality_label = args
    
    audio = load_audio(path)
    if audio is None:
        return []
    
    results = []
    augmented_audios = augment_audio(audio, SAMPLE_RATE)
    
    for aug_audio in augmented_audios:
        spec = audio_to_spectrogram(aug_audio)
        results.append((spec, distance_label, quality_label))
    
    return results

print("✅ Spectrogram functions loaded")

In [ ]:
# === CELL 7: Multi-Task CNN Model ===
import torch
import torch.nn as nn
import torch.nn.functional as F

class DistanceQualityCNN(nn.Module):
    """
    Multi-task CNN for distance and quality classification.
    Deep architecture optimized for A100.
    """
    def __init__(self, num_distance_classes=5, num_quality_classes=5):
        super().__init__()
        
        # Shared feature extractor - 5 conv blocks
        self.features = nn.Sequential(
            # Block 1: 128x128 -> 64x64
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.1),
            
            # Block 2: 64x64 -> 32x32
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.1),
            
            # Block 3: 32x32 -> 16x16
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2),
            
            # Block 4: 16x16 -> 8x8
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2),
            
            # Block 5: 8x8 -> 4x4
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.3),
        )
        
        # Shared dense layers
        self.shared_fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
        )
        
        # Distance head
        self.distance_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_distance_classes)
        )
        
        # Quality head
        self.quality_head = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_quality_classes)
        )
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        features = self.features(x)
        shared = self.shared_fc(features)
        
        distance_out = self.distance_head(shared)
        quality_out = self.quality_head(shared)
        
        return distance_out, quality_out

# Test model
model = DistanceQualityCNN().to(device)
dummy = torch.randn(1, 1, 128, 128).to(device)
dist_out, qual_out = model(dummy)
print(f"✅ Model created")
print(f"   Distance output: {dist_out.shape}")
print(f"   Quality output: {qual_out.shape}")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

del model, dummy
torch.cuda.empty_cache()

In [ ]:
# === CELL 8: Training Dataset Class ===
from torch.utils.data import Dataset, DataLoader

class DistanceQualityDataset(Dataset):
    """Dataset for distance/quality training."""
    
    def __init__(self, spectrograms, distance_labels, quality_labels):
        self.spectrograms = torch.FloatTensor(spectrograms).unsqueeze(1)
        self.distance_labels = torch.LongTensor(distance_labels)
        self.quality_labels = torch.LongTensor(quality_labels)
    
    def __len__(self):
        return len(self.spectrograms)
    
    def __getitem__(self, idx):
        return (
            self.spectrograms[idx],
            self.distance_labels[idx],
            self.quality_labels[idx]
        )

print("✅ Dataset class defined")

In [ ]:
# === CELL 9: Process Audio Files (PARALLEL) ===
from tqdm.notebook import tqdm
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing as mp

# Label encoding
distance_to_idx = {c: i for i, c in enumerate(DISTANCE_CLASSES)}
quality_to_idx = {c: i for i, c in enumerate(QUALITY_CLASSES)}

print(f"Distance classes: {distance_to_idx}")
print(f"Quality classes: {quality_to_idx}")

# Check if cached spectrograms exist
cache_path = CACHE_DIR / 'preprocessed_spectrograms.npz'

if cache_path.exists():
    print(f"\n📂 Loading cached spectrograms...")
    data = np.load(cache_path)
    X = data['spectrograms']
    y_distance = data['distance_labels']
    y_quality = data['quality_labels']
    print(f"✅ Loaded {len(X):,} spectrograms from cache")
else:
    print(f"\n🔄 Processing audio files from Google Drive (PARALLEL)...")
    
    # Build list of files to process
    valid_rows = []
    for idx, row in df.iterrows():
        if pd.isna(row['audio_file']):
            continue
        audio_path = AUDIO_DIR / row['audio_file']
        if not audio_path.exists():
            continue
        dist_label = distance_to_idx.get(row['distance_category'])
        qual_label = quality_to_idx.get(row['quality_category'])
        if dist_label is not None and qual_label is not None:
            valid_rows.append((audio_path, dist_label, qual_label))
    
    print(f"Valid audio files: {len(valid_rows):,}")
    
    # Parallel processing - use most CPUs
    num_workers = min(mp.cpu_count() - 1, 8)  # Leave 1 CPU free, max 8
    print(f"Using {num_workers} parallel workers\n")
    
    all_spectrograms = []
    all_distance_labels = []
    all_quality_labels = []
    
    # Process in parallel with progress bar
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        # Submit all tasks
        futures = {executor.submit(process_audio_file, args): args for args in valid_rows}
        
        # Collect results with progress bar
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing audio"):
            try:
                results = future.result()
                for spec, dist_label, qual_label in results:
                    all_spectrograms.append(spec)
                    all_distance_labels.append(dist_label)
                    all_quality_labels.append(qual_label)
            except Exception as e:
                pass  # Skip failed files silently
    
    X = np.array(all_spectrograms)
    y_distance = np.array(all_distance_labels)
    y_quality = np.array(all_quality_labels)
    
    # Cache for next run
    print(f"\n💾 Saving to cache...")
    np.savez_compressed(cache_path, 
                        spectrograms=X, 
                        distance_labels=y_distance, 
                        quality_labels=y_quality)
    
    print(f"\n✅ Processed {len(X):,} spectrograms (with augmentation)")

print(f"\n📊 Dataset shape: {X.shape}")

In [ ]:
# === CELL 10: Train/Val/Test Split ===
from sklearn.model_selection import train_test_split

# First split: train+val vs test (90% / 10%)
X_trainval, X_test, y_dist_trainval, y_dist_test, y_qual_trainval, y_qual_test = train_test_split(
    X, y_distance, y_quality, test_size=0.1, random_state=42, stratify=y_distance
)

# Second split: train vs val (80% / 20% of trainval = 72% / 18% of total)
X_train, X_val, y_dist_train, y_dist_val, y_qual_train, y_qual_val = train_test_split(
    X_trainval, y_dist_trainval, y_qual_trainval, test_size=0.2, random_state=42, stratify=y_dist_trainval
)

print(f"📊 Data splits:")
print(f"   Train: {len(X_train):,} ({100*len(X_train)/len(X):.0f}%)")
print(f"   Val:   {len(X_val):,} ({100*len(X_val)/len(X):.0f}%)")
print(f"   Test:  {len(X_test):,} ({100*len(X_test)/len(X):.0f}%)")

# Create datasets
train_dataset = DistanceQualityDataset(X_train, y_dist_train, y_qual_train)
val_dataset = DistanceQualityDataset(X_val, y_dist_val, y_qual_val)
test_dataset = DistanceQualityDataset(X_test, y_dist_test, y_qual_test)

# Create dataloaders
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=4, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=4, pin_memory=True
)

print(f"\n✅ DataLoaders created")

In [ ]:
# === CELL 11: Training Loop ===
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import time
from datetime import datetime

# Model, loss, optimizer
model = DistanceQualityCNN(
    num_distance_classes=len(DISTANCE_CLASSES),
    num_quality_classes=len(QUALITY_CLASSES)
).to(device)

criterion_distance = nn.CrossEntropyLoss()
criterion_quality = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

# Cosine annealing with warm restarts
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=MIN_LR)

# Training tracking
best_val_acc = 0
best_model_state = None
patience_counter = 0
history = {'train_loss': [], 'val_loss': [], 'val_dist_acc': [], 'val_qual_acc': []}

print(f"{'='*60}")
print(f"🚀 STARTING TRAINING")
print(f"{'='*60}")
print(f"Start: {datetime.now().strftime('%H:%M:%S')}")
print(f"Epochs: {EPOCHS} | Patience: {PATIENCE}")
print(f"Batch size: {BATCH_SIZE} | LR: {LEARNING_RATE}")
print(f"{'='*60}\n")

start_time = time.time()

for epoch in range(EPOCHS):
    epoch_start = time.time()
    
    # === TRAINING ===
    model.train()
    train_loss = 0
    
    for specs, dist_labels, qual_labels in train_loader:
        specs = specs.to(device, non_blocking=True)
        dist_labels = dist_labels.to(device, non_blocking=True)
        qual_labels = qual_labels.to(device, non_blocking=True)
        
        optimizer.zero_grad()
        
        dist_out, qual_out = model(specs)
        
        # Combined loss (weighted)
        loss_dist = criterion_distance(dist_out, dist_labels)
        loss_qual = criterion_quality(qual_out, qual_labels)
        loss = 0.6 * loss_dist + 0.4 * loss_qual  # Distance is primary task
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    scheduler.step()
    train_loss /= len(train_loader)
    
    # === VALIDATION ===
    model.eval()
    val_loss = 0
    dist_correct = 0
    qual_correct = 0
    total = 0
    
    with torch.no_grad():
        for specs, dist_labels, qual_labels in val_loader:
            specs = specs.to(device, non_blocking=True)
            dist_labels = dist_labels.to(device, non_blocking=True)
            qual_labels = qual_labels.to(device, non_blocking=True)
            
            dist_out, qual_out = model(specs)
            
            loss_dist = criterion_distance(dist_out, dist_labels)
            loss_qual = criterion_quality(qual_out, qual_labels)
            loss = 0.6 * loss_dist + 0.4 * loss_qual
            val_loss += loss.item()
            
            dist_correct += (dist_out.argmax(1) == dist_labels).sum().item()
            qual_correct += (qual_out.argmax(1) == qual_labels).sum().item()
            total += dist_labels.size(0)
    
    val_loss /= len(val_loader)
    val_dist_acc = dist_correct / total
    val_qual_acc = qual_correct / total
    combined_acc = (val_dist_acc + val_qual_acc) / 2
    
    # Save history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_dist_acc'].append(val_dist_acc)
    history['val_qual_acc'].append(val_qual_acc)
    
    # Check for improvement
    if combined_acc > best_val_acc:
        best_val_acc = combined_acc
        best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        patience_counter = 0
        marker = '✅ NEW BEST'
    else:
        patience_counter += 1
        marker = ''
    
    epoch_time = time.time() - epoch_start
    lr = optimizer.param_groups[0]['lr']
    
    print(f"Epoch {epoch+1:3d}/{EPOCHS} | "
          f"Loss: {train_loss:.4f}/{val_loss:.4f} | "
          f"Dist: {val_dist_acc:.1%} | Qual: {val_qual_acc:.1%} | "
          f"LR: {lr:.6f} | {epoch_time:.0f}s {marker}")
    
    # Early stopping
    if patience_counter >= PATIENCE:
        print(f"\n⏹️ Early stopping at epoch {epoch+1}")
        break

total_time = time.time() - start_time
print(f"\n{'='*60}")
print(f"🏁 TRAINING COMPLETE")
print(f"{'='*60}")
print(f"Time: {total_time/60:.1f} minutes")
print(f"Best combined accuracy: {best_val_acc:.1%}")

In [ ]:
# === CELL 12: Evaluate on Test Set ===
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load best model
model.load_state_dict(best_model_state)
model.eval()

# Test evaluation
all_dist_preds = []
all_dist_true = []
all_qual_preds = []
all_qual_true = []

with torch.no_grad():
    for specs, dist_labels, qual_labels in test_loader:
        specs = specs.to(device)
        
        dist_out, qual_out = model(specs)
        
        all_dist_preds.extend(dist_out.argmax(1).cpu().numpy())
        all_dist_true.extend(dist_labels.numpy())
        all_qual_preds.extend(qual_out.argmax(1).cpu().numpy())
        all_qual_true.extend(qual_labels.numpy())

# Distance report
print(f"\n{'='*60}")
print("📊 DISTANCE CLASSIFICATION REPORT")
print(f"{'='*60}")
print(classification_report(all_dist_true, all_dist_preds, target_names=DISTANCE_CLASSES))

# Quality report
print(f"\n{'='*60}")
print("📊 QUALITY CLASSIFICATION REPORT")
print(f"{'='*60}")
print(classification_report(all_qual_true, all_qual_preds, target_names=QUALITY_CLASSES))

# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distance confusion matrix
cm_dist = confusion_matrix(all_dist_true, all_dist_preds)
sns.heatmap(cm_dist, annot=True, fmt='d', cmap='Blues',
            xticklabels=DISTANCE_CLASSES, yticklabels=DISTANCE_CLASSES, ax=axes[0])
axes[0].set_title('Distance Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')

# Quality confusion matrix
cm_qual = confusion_matrix(all_qual_true, all_qual_preds)
sns.heatmap(cm_qual, annot=True, fmt='d', cmap='Greens',
            xticklabels=QUALITY_CLASSES, yticklabels=QUALITY_CLASSES, ax=axes[1])
axes[1].set_title('Quality Confusion Matrix')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('True')

plt.tight_layout()
plt.savefig(MODELS_DIR / 'confusion_matrices.png', dpi=150)
plt.show()

In [ ]:
# === CELL 13: Training History Plot ===
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy plot
axes[1].plot(history['val_dist_acc'], label='Distance Acc')
axes[1].plot(history['val_qual_acc'], label='Quality Acc')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig(MODELS_DIR / 'training_history.png', dpi=150)
plt.show()

In [ ]:
# === CELL 14: Save Model ===
from sklearn.metrics import accuracy_score
import os

dist_acc = accuracy_score(all_dist_true, all_dist_preds)
qual_acc = accuracy_score(all_qual_true, all_qual_preds)

model_path = MODELS_DIR / f'distance_quality_cnn_{VERSION}.pt'

torch.save({
    'model_state_dict': best_model_state,
    'distance_classes': DISTANCE_CLASSES,
    'quality_classes': QUALITY_CLASSES,
    'distance_nl': DISTANCE_NL,
    'distance_accuracy': dist_acc,
    'quality_accuracy': qual_acc,
    'version': VERSION,
    'training_config': {
        'epochs': len(history['train_loss']),
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'augmentation': USE_AUGMENTATION,
        'n_mels': N_MELS,
        'sample_rate': SAMPLE_RATE
    },
    'history': history
}, model_path)

print(f"\n{'='*60}")
print(f"✅ MODEL SAVED")
print(f"{'='*60}")
print(f"Path: {model_path}")
print(f"Size: {model_path.stat().st_size / 1e6:.1f} MB")
print(f"Distance accuracy: {dist_acc:.1%}")
print(f"Quality accuracy: {qual_acc:.1%}")

In [ ]:
# === CELL 15: Save Model to Google Drive ===
import shutil

# Copy model to Google Drive for persistence
drive_models_dir = DRIVE_DATA_DIR / 'models'
drive_models_dir.mkdir(parents=True, exist_ok=True)

drive_model_path = drive_models_dir / f'distance_quality_cnn_{VERSION}.pt'
shutil.copy(model_path, drive_model_path)

# Also copy plots
shutil.copy(MODELS_DIR / 'confusion_matrices.png', drive_models_dir / 'confusion_matrices.png')
shutil.copy(MODELS_DIR / 'training_history.png', drive_models_dir / 'training_history.png')

print(f"✅ Model saved to Google Drive: {drive_model_path}")
print(f"\nNa training, download het model via:")
print(f"   Drive → EMSN → distance_quality_training → models")

In [ ]:
# === CELL 16: Download Model (Optional) ===
from google.colab import files
import shutil

# Create ZIP with model and images
print("📦 Creating download package...")

zip_name = f'emsn_distance_quality_{VERSION}'
shutil.make_archive(f'/content/{zip_name}', 'zip', MODELS_DIR)

zip_path = f'/content/{zip_name}.zip'
zip_size = os.path.getsize(zip_path) / 1e6

print(f"\n✅ Package ready: {zip_path}")
print(f"Size: {zip_size:.1f} MB")

print("\n📥 Starting download...")
files.download(zip_path)

## Na het downloaden

Upload het model naar je Pi:
```bash
# Op je PC
scp emsn_distance_quality_2026_ultimate.zip ronny@192.168.1.178:~/

# Op de Pi
unzip emsn_distance_quality_2026_ultimate.zip -d ~/emsn2/models/
```

Of download direct van Google Drive:
```bash
# Op de Pi met rclone
rclone copy gdrive:/EMSN/distance_quality_training/models/ ~/emsn2/models/
```

Het model kan dan gebruikt worden door de quality_enricher voor real-time classificatie.